--- Day 6: Guard Gallivant ---
The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

In [112]:
guard_characters = ["^", ">", "V", "<"]
grid = [
    "....#.....",
    ".........#",
    "..........",
    "..#.......",
    ".......#..",
    "..........",
    ".#..^.....",
    "........#.",
    "#.........",
    "......#...",
]
guard_loc_x = 0
guard_loc_y = 0
guard_direction = "^"

In [113]:
def get_file_content(grid):
    with open("day-6-input.txt", "r") as file:
        # Read all lines and strip any trailing newlines or spaces
        grid = [line.strip() for line in file]
    return grid

In [114]:
for y in range(len(grid)):
    for x in range(len(grid[y])):
        if grid[y][x] in guard_characters:
            guard_loc_x = x
            guard_loc_y = y
            guard_direction = grid[y][x]


def get_movement_dir(guard_dir):
    match guard_dir:
        case "^":
            return (0, -1)
        case ">":
            return (1, 0)
        case "V":
            return (0, 1)
        case "<":
            return (-1, 0)
        case _:
            return (0, 0)


def get_lookahead(guard_loc_x, guard_loc_y, offset, grid):
    try:
        # Attempt to access the list at the given index
        value = grid[guard_loc_y + offset[1]][guard_loc_x + offset[0]]
        return value != "#"
    except IndexError:
        # Handle out-of-range error
        raise Exception("End Of Path")


def get_next_direction(guard_direction):
    index_of_direction = guard_characters.index(guard_direction)
    try:
        return guard_characters[index_of_direction + 1]
    except IndexError:
        return guard_characters[0]


def change_to_X(guard_loc_x, guard_loc_y, grid):
    mutable_grid = [list(row) for row in grid]
    mutable_grid[guard_loc_y][guard_loc_x] = "X"
    grid = ["".join(row) for row in mutable_grid]
    return grid


def move_forward(guard_loc_x, guard_loc_y, guard_direction, grid):
    offset = (0, 0)
    can_move_forward = False
    while can_move_forward == False:
        offset = get_movement_dir(guard_direction)
        can_move_forward = get_lookahead(guard_loc_x, guard_loc_y, offset, grid)
        if can_move_forward == False:
            guard_direction = get_next_direction(guard_direction)
    grid = change_to_X(guard_loc_x, guard_loc_y, grid)
    guard_loc_y = guard_loc_y + offset[1]
    guard_loc_x = guard_loc_x + offset[0]
    return (guard_loc_x, guard_loc_y, guard_direction, grid)

The map shows the current position of the guard with ^ (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

If there is something directly in front of you, turn right 90 degrees.
Otherwise, take a step forward.

In [115]:
grid = get_file_content(grid)
while True:
    try:
        guard_loc_x, guard_loc_y, guard_direction, grid = move_forward(
            guard_loc_x, guard_loc_y, guard_direction, grid
        )
    except Exception as e:
        grid = change_to_X(guard_loc_x, guard_loc_y, grid)
        print("End of path")
        break

end_count = 0


for y in range(len(grid)):
    print(grid[y])
    for x in range(len(grid[y])):
        if grid[y][x] == "X":
            end_count += 1

print(end_count)

End of path
....X...X..#....................#......#.....................#.#...........#......................................................
....X...X............................#....#.................#...............................#.....................................
..#.X#..X....#.............................................#.#.........................#......................#..............#....
....X...X................#.....#...............#.................#................................................................
....X..#X.........#........#.#............................................#....#.....................#..........##..#.............
....X...X#.....................................................#..............................................................#...
....X#..X...........................................................#................##.....#...........#.......#.........#.......
.......#X........#......#...............#............................#.